In [2]:
!pip install joblib

    100% |████████████████████████████████| 286kB 5.2MB/s eta 0:00:011


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
import seaborn as sns
import random

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_fscore_support

import joblib

In [4]:
#X_train
filename = "X_train.pkl"
X_train_all = pd.read_pickle(filename)#, skiprows=skip, sep='\t',index_col=0)

#X_test
filename = "X_test.pkl"
X_test_all = pd.read_pickle(filename)#, skiprows=skip, sep='\t',index_col=0)

#y_train
filename = "y_train.pkl"
y_train_all = pd.read_pickle(filename)#, skiprows=skip, sep='\t',index_col=0)

#y_test
filename = "y_test.pkl"
y_test_all = pd.read_pickle(filename)

In [5]:
#drop OHE source columns & unuseful categorical variables
X_train_all.drop(columns=['term','verification_status',
                          'grade','emp_title', 'addr_state',
                          #ALSO, drop redundant columns that new OHE columns provide the info for
                          'debt_settlement_flag',#ALSO, drop columns clearly not predictive of class
                         ],inplace=True) #ALSO, drop date columns

In [6]:
#REPEAT DROPS for X_test
X_test_all.drop(columns=['term','verification_status',
                         'grade','emp_title', 'addr_state',
                         #ALSO, drop redundant columns that new OHE columns provide the info for
                         'debt_settlement_flag',#ALSO, drop columns clearly not predictive of class
                        ],inplace=True) #ALSO, drop date columns

In [16]:
X_train_all.set_index('index',inplace=True)
X_test_all.set_index('index',inplace=True)

### Create Regression Target Variable: Annualized Return

In [12]:
y_train_all['annu_return'] = (X_train_all['total_pymnt'] / 
                              X_train_all['funded_amnt']) ** (365.0 / 
                                                              ((pd.to_datetime(X_train_all['last_pymnt_d']) - 
                                                                pd.to_datetime(X_train_all['issue_d'])).dt.days + 30))
y_test_all['annu_return'] = (X_test_all['total_pymnt'] / 
                              X_test_all['funded_amnt']) ** (365.0 / 
                                                             ((pd.to_datetime(X_test_all['last_pymnt_d']) - 
                                                               pd.to_datetime(X_test_all['issue_d'])).dt.days + 30))

In [19]:
X_train_all[['total_pymnt','funded_amnt','last_pymnt_d','issue_d']].head(20)

,total_pymnt,funded_amnt,last_pymnt_d,issue_d
index,,,,
798015,24704.737119,20000.0,May-2017,Feb-2015
962964,10780.580000,15000.0,May-2016,Jun-2014
534392,18049.571386,16000.0,Jan-2019,Nov-2015
668134,27124.598857,24000.0,Jul-2018,Jul-2015
1068586,26527.946309,24000.0,Nov-2016,Nov-2013
677314,3510.930000,15000.0,Sep-2015,Jul-2015
16225,4346.656209,4000.0,Oct-2018,Nov-2017
1142972,54000.347778,35000.0,Mar-2017,Apr-2013
1156706,10799.289665,9600.0,Mar-2014,Mar-2013


In [30]:
X_test_all['issue_d'].head()

index
224551     Oct-2016
1179835    Nov-2012
386096     Mar-2016
579196     Oct-2015
1122573    Jun-2013
Name: issue_d, dtype: object

In [14]:
y_train_all.head(20)

,loan_status,annu_return
798015,1,1.069937
962964,0,NaN
534392,1,0.839323
668134,1,1.056104
1068586,1,NaN
677314,0,1.053596
16225,1,1.076653
1142972,1,NaN
1156706,1,NaN
103346,0,1.068300
